In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import re
import operator
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import pydicom

from pymedphys_analysis.tpscompare import absolute_scans_from_mephysto
from pymedphys.dicom import 

In [ ]:
DICOM_DIR = next(Path(r"S:\Physics\Monaco\Model vs Measurement Comparisons\Beam Models").glob(r"External *\DICOM Dose Exports"))
MEASUREMENTS_DIR = Path(r"S:\Physics\Monaco\Model vs Measurement Comparisons\Measurements\RCCC\Photons\With Flattening Filter")

In [ ]:
mephysto_files = list(MEASUREMENTS_DIR.glob('06MV * Open.mcc'))
keys = [
    re.match('06MV (\d\dx\d\d) Open.mcc', filepath.name).group(1)
    for filepath in mephysto_files
]

mephysto_file_map = {
    key: filepath for key, filepath in zip(keys, mephysto_files)
}

mephysto_file_map

In [ ]:
dicom_file_map = {
    key: DICOM_DIR.joinpath(f'06MV_{key}.dcm')
    for key in keys
}

dicom_file_map

In [ ]:
dicom_dataset_map = {
    key: pydicom.read_file(str(dicom_file_map[key]), force=True)
    for key in keys
}

In [ ]:
absolute_dose_table = pd.read_csv(MEASUREMENTS_DIR.joinpath('AbsoluteDose.csv'), index_col=0)
absolute_dose = absolute_dose_table['d10 @ 90 SSD']['6 MV']
absolute_dose

In [ ]:
output_factors = pd.read_csv(MEASUREMENTS_DIR.joinpath('OutputFactors.csv'), index_col=0)
output_factors

In [ ]:
absolute_dose_per_field = {
    key: output_factors[key]['6 MV'] * absolute_dose
    for key in keys
}

absolute_dose_per_field

In [ ]:
absolute_scans_per_field = {
    key: absolute_scans_from_mephysto(
        mephysto_file_map[key], 
        absolute_dose_per_field[key], 100)
    for key in keys
}

# absolute_scans_per_field

In [ ]:
getter = operator.itemgetter('displacement', 'dose')

In [ ]:
for key in keys:
    plt.plot(*getter(absolute_scans_per_field[key]['depth_dose']))

In [ ]:
depths = absolute_scans_per_field['02x02']['profiles'].keys()
for depth in depths:
    plt.plot(*getter(absolute_scans_per_field['02x02']['profiles'][depth]['inplane']), label=f'Depth: {depth} mm')
    
plt.legend()

In [ ]:
depths = absolute_scans_per_field['02x02']['profiles'].keys()
for depth in depths:
    plt.plot(*getter(absolute_scans_per_field['02x02']['profiles'][depth]['crossplane']), label=f'Depth: {depth} mm')
    
plt.legend()